<a href="https://colab.research.google.com/github/gurtaransingh/Parameter_Optimization/blob/main/Parameter_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importing Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import learning_curve
import warnings
warnings.filterwarnings('ignore')

###Uploading the Dataset from UCI Library

In [11]:
from google.colab import files
uploaded = files.upload()

Saving Live_20210128.csv to Live_20210128.csv


In [12]:
dataset = pd.read_csv('/content/Live_20210128.csv')
dataset.head()

status_id status_type status_published  num_reactions  num_comments  \
0          1       video   4/22/2018 6:00            529           512   
1          2       photo  4/21/2018 22:45            150             0   
2          3       video   4/21/2018 6:17            227           236   
3          4       photo   4/21/2018 2:29            111             0   
4          5       photo   4/18/2018 3:22            213             0   

   num_shares  num_likes  num_loves  num_wows  num_hahas  num_sads  \
0         262        432         92         3          1         1   
1           0        150          0         0          0         0   
2          57        204         21         1          1         0   
3           0        111          0         0          0         0   
4           0        204          9         0          0         0   

   num_angrys  Column1  Column2  Column3  Column4  
0           0      NaN      NaN      NaN      NaN  
1           0      NaN      NaN      NaN      NaN  
2           0      NaN      NaN      NaN      NaN  
3           0      NaN      NaN      NaN      NaN  
4           0      NaN      NaN      NaN      NaN

###Pre-Processing of Dataset

In [13]:
dataset.shape

(7050, 16)

In [14]:
dataset.isnull().sum()

status_id              0
status_type            0
status_published       0
num_reactions          0
num_comments           0
num_shares             0
num_likes              0
num_loves              0
num_wows               0
num_hahas              0
num_sads               0
num_angrys             0
Column1             7050
Column2             7050
Column3             7050
Column4             7050
dtype: int64

In [15]:
dataset = dataset.drop(['status_id','status_published','Column1','Column2','Column3','Column4'],axis=1)

In [16]:
sns.countplot(x = 'status_type', data=dataset)

<Axes: xlabel='status_type', ylabel='count'>

In [17]:
dataset.head()

status_type  num_reactions  num_comments  num_shares  num_likes  num_loves  \
0       video            529           512         262        432         92   
1       photo            150             0           0        150          0   
2       video            227           236          57        204         21   
3       photo            111             0           0        111          0   
4       photo            213             0           0        204          9   

   num_wows  num_hahas  num_sads  num_angrys  
0         3          1         1           0  
1         0          0         0           0  
2         1          1         0           0  
3         0          0         0           0  
4         0          0         0           0

In [21]:
X = dataset.iloc[:,1:]

y = dataset['status_type']

In [22]:
X

num_reactions  num_comments  num_shares  num_likes  num_loves  num_wows  \
0               529           512         262        432         92         3   
1               150             0           0        150          0         0   
2               227           236          57        204         21         1   
3               111             0           0        111          0         0   
4               213             0           0        204          9         0   
...             ...           ...         ...        ...        ...       ...   
7045             89             0           0         89          0         0   
7046             16             0           0         14          1         0   
7047              2             0           0          1          1         0   
7048            351            12          22        349          2         0   
7049             17             0           0         17          0         0   

      num_hahas  num_sads  num_angrys  
0             1         1           0  
1             0         0           0  
2             1         0           0  
3             0         0           0  
4             0         0           0  
...         ...       ...         ...  
7045          0         0           0  
7046          1         0           0  
7047          0         0           0  
7048          0         0           0  
7049          0         0           0  

[7050 rows x 9 columns]

In [23]:
ss = StandardScaler()
ss.fit_transform(X)

array([[ 0.64610394,  0.32335035,  1.6868788 , ...,  0.07671305,
         0.47357035, -0.15574803],
       [-0.17319166, -0.25220622, -0.3041444 , ..., -0.17600986,
        -0.152587  , -0.15574803],
       [-0.00673846,  0.01308938,  0.12901714, ...,  0.07671305,
        -0.152587  , -0.15574803],
       ...,
       [-0.49312768, -0.25220622, -0.3041444 , ..., -0.17600986,
        -0.152587  , -0.15574803],
       [ 0.26131603, -0.23871662, -0.13695924, ..., -0.17600986,
        -0.152587  , -0.15574803],
       [-0.46070173, -0.25220622, -0.3041444 , ..., -0.17600986,
        -0.152587  , -0.15574803]])

###Creation of 10 samples with 70-30 ratio of Training and Testing Set

In [24]:
samples = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)
    samples.append((X_train, X_test, y_train, y_test))

In [25]:
samples

[(      num_reactions  num_comments  num_shares  num_likes  num_loves  num_wows  \
  1231           3190            19           1       3190          0         0   
  3312             10             0           0         10          0         0   
  2696             37             0           2         35          1         1   
  5849              8             0           0          7          1         0   
  4929             20             0           0         20          0         0   
  ...             ...           ...         ...        ...        ...       ...   
  4931             34             0           0         33          0         1   
  3264            296           869          58        252         39         2   
  1653            115             9           0        115          0         0   
  2607             25             9           0         25          0         0   
  2732            174            66           1        173          1         0   
  
 

### Creation of a Fitness Function

In [26]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
result = pd.DataFrame(columns=['Sample', 'Best Accuracy', 'Best Kernel', 'Best Nu', 'Best Epsilon'])

In [27]:
def fitnessFunction(kernel, C, gamma):
  svm = SVC(kernel=kernel, C=C, gamma=gamma, max_iter=1000)
  svm.fit(X_train, y_train)
  y_pred = svm.predict(X_test)
  return accuracy_score(y_pred, y_test)

### Calling the Function for Every Sample

In [28]:
for i in range(len(samples)):
  best_accuracy = 0
  best_C = 0
  best_gamma = 0
  for kernel in kernels:
    X_train, X_test, y_train, y_test = samples[i]
    C = np.random.uniform(0, 10)
    gamma = np.random.uniform(0, 10)
    score = fitnessFunction(kernel, C, gamma)
    if score>best_accuracy:
      best_accuracy = round(score, 2)
      best_C = round(C, 2)
      best_gamma = round(gamma, 2)
      best_kernel = kernel
  print('Best Accuracy = ', best_accuracy, 'Best Kernel = ', best_kernel, 'Best Nu = ', best_C, 'Best Epsilon = ', best_gamma)
  result.loc[i] = [i+1, best_accuracy, best_kernel, best_C, best_gamma]

Best Accuracy =  0.63 Best Kernel =  poly Best Nu =  0.54 Best Epsilon =  0.25
Best Accuracy =  0.62 Best Kernel =  rbf Best Nu =  7.25 Best Epsilon =  5.11
Best Accuracy =  0.66 Best Kernel =  poly Best Nu =  0.19 Best Epsilon =  2.5
Best Accuracy =  0.42 Best Kernel =  rbf Best Nu =  1.64 Best Epsilon =  7.45
Best Accuracy =  0.44 Best Kernel =  rbf Best Nu =  4.52 Best Epsilon =  9.81
Best Accuracy =  0.72 Best Kernel =  sigmoid Best Nu =  6.71 Best Epsilon =  1.0
Best Accuracy =  0.5 Best Kernel =  rbf Best Nu =  9.24 Best Epsilon =  7.97
Best Accuracy =  0.66 Best Kernel =  sigmoid Best Nu =  3.41 Best Epsilon =  3.43
Best Accuracy =  0.66 Best Kernel =  poly Best Nu =  6.39 Best Epsilon =  4.73
Best Accuracy =  0.63 Best Kernel =  rbf Best Nu =  2.99 Best Epsilon =  9.72


###Creation of a Result Table

In [29]:
result

Sample  Best Accuracy Best Kernel  Best Nu  Best Epsilon
0       1           0.63        poly     0.54          0.25
1       2           0.62         rbf     7.25          5.11
2       3           0.66        poly     0.19          2.50
3       4           0.42         rbf     1.64          7.45
4       5           0.44         rbf     4.52          9.81
5       6           0.72     sigmoid     6.71          1.00
6       7           0.50         rbf     9.24          7.97
7       8           0.66     sigmoid     3.41          3.43
8       9           0.66        poly     6.39          4.73
9      10           0.63         rbf     2.99          9.72

### Plotting of the Convergence Graph/Linear Curve

In [30]:
X_train, X_test, y_train, y_test = samples[result['Best Accuracy'].idxmax()]

In [31]:
train_sizes, train_scores, test_scores = learning_curve(SVC(kernel=result['Best Kernel'].iloc[result['Best Accuracy'].idxmax()], 
                                                        C=result['Best Nu'].iloc[result['Best Accuracy'].idxmax()],
                                                        gamma=result['Best Epsilon'].iloc[result['Best Accuracy'].idxmax()],
                                                        max_iter = 1000), X_train, y_train, cv=10, scoring='accuracy', n_jobs=-1,
                                                        train_sizes = np.linspace(0.01, 1.0, 50))

In [32]:
train_sizes

array([  44,  134,  223,  313,  403,  493,  582,  672,  762,  851,  941,
       1031, 1121, 1210, 1300, 1390, 1480, 1569, 1659, 1749, 1838, 1928,
       2018, 2108, 2197, 2287, 2377, 2467, 2556, 2646, 2736, 2825, 2915,
       3005, 3095, 3184, 3274, 3364, 3454, 3543, 3633, 3723, 3812, 3902,
       3992, 4082, 4171, 4261, 4351, 4441])

In [33]:
train_mean = np.mean(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)

In [34]:
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Training Score')
plt.plot(train_sizes, test_mean, label='Cross-Validation Score')
plt.xlabel('Iterations')
plt.ylabel('Accuracy')
plt.title('Convergence Graph')
plt.legend(loc="best")
plt.show()